In [ ]:
!pip install numpy==1.16.0
!pip install tensorflow==2.3.1 --user

In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras.models import load_model

In [2]:
tf.__version__

'2.3.1'

In [3]:
model = load_model('Models\model_40x40.hdf5')
IMG_WIDTH = IMG_HEIGHT = 750

In [4]:
# cap = cv.VideoCapture(0)
# cap.set(3, 640)
# cap.set(4, 480)

In [5]:
def biggestContour(contours):
    """Finds the biggest contour in the image"""
    max_area = 0
    for contour in contours:
        area = cv.contourArea(contour)
        perimeter = cv.arcLength(contour, True)
        vertices = cv.approxPolyDP(contour, 0.02*perimeter, True)
        if area > max_area and len(vertices == 4):
            max_area = area
            biggest = vertices
    return biggest

In [6]:
def getBinary(img, biggest):
#     peri = cv.arcLength(biggest, True)
#     approx = cv.approxPolyDP(biggest, 0.02*peri, True)
    ax = biggest[0][0]
    ay = biggest[1][0]
    bx = biggest[2][0]
    by = biggest[3][0]
    cx = biggest[4][0]
    cy = biggest[5][0]
    dx = biggest[6][0]
    dy = biggest[7][0]

    width, height = IMG_WIDTH, IMG_HEIGHT
    pts1 = np.float32([[ax, ay], [bx, by], [cx, cy], [dx, dy]])
    pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])

    matrix = cv.getPerspectiveTransform(pts1, pts2)
    img_perspective = cv.warpPerspective(img, matrix, (width, height))
    img_corners = cv.cvtColor(img_perspective, cv.COLOR_BGR2GRAY)

    thresh, img_binary = cv.threshold(img_corners, 125, 255, cv.THRESH_BINARY)
    return img_binary, pts1, pts2

In [7]:
def reorder(myPoints):
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] =myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
    return myPointsNew

In [8]:
def possible(grid, x, y, n):
    """To check whether n can be inserted at position (x, y)."""
    for i in range(9):
        if grid[x][i] == n:
            return False
    for i in range(9):
        if grid[i][y] == n:
            return False
    for i in range(int(x/3) * 3, int(x/3) * 3 + 3):
        for j in range(int(y/3) * 3, int(y/3) * 3 + 3):
            if grid[i][j] == n:
                return False
    return True

def findNextCell(grid, x, y):
        for i in range(x,9):
                for j in range(y,9):
                        if grid[i][j] == 0:
                                return i, j
        for i in range(0,9):
                for j in range(0,9):
                        if grid[i][j] == 0:
                                return i, j
        return -1,-1

def solveSudoku(grid, x=0, y=0):
        x, y = findNextCell(grid, x, y)
        if x == -1:
                return True
        for n in range(1,10):
                if possible(grid, x, y, n):
                        grid[x][y] = n
                        if solveSudoku(grid, x, y):
                                return True
                        grid[x][y] = 0
        return False

In [9]:
def identifyDigit(img, img_cropped):
#     print(cv.countNonZero(img))
    """Identifies the digit from the cropped image"""
    if cv.countNonZero(img_cropped) > 1550:
        return 0
    else:
        return model.predict(np.array(img).reshape(1, 40, 40,1)).argmax() 

In [37]:
def img2grid(img_binary, grid):
    """Converts the image to a sudoku grid"""
    side = 8
    for i in range (9):
        for j in range (9):
            img_cropped = img_binary[int(i*IMG_WIDTH/9 + side):int(i*IMG_WIDTH/9 + IMG_WIDTH/9 - side), int(j*IMG_WIDTH/9 + side): int(j*IMG_WIDTH/9 + IMG_WIDTH/9 - side)]
            img_cropped = cv.resize(img_cropped, (40, 40))
            grid[i][j] = identifyDigit(img_cropped, cv.erode(img_cropped, np.ones((1, 1),np.uint8), iterations = 1))
    return

In [11]:
def showAnswer(grid, color = (0, 102, 0)):
    imgSolved = np.zeros((IMG_WIDTH, IMG_HEIGHT, 3))
    side = 20
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                cv.putText(imgSolved, str(int(grid[i][j])), 
                           (int(j*IMG_WIDTH/9 + side), int((i+1)*IMG_WIDTH/9 - side)), 
                           cv.FONT_HERSHEY_SIMPLEX, 2.1, color, 3)
    for i in range(10):
        cv.line(imgSolved, (int(i*IMG_WIDTH/9), 0), (int(i*IMG_WIDTH/9), IMG_WIDTH), color, 5, 1)
        cv.line(imgSolved, (0, int(i*IMG_WIDTH/9)), (IMG_WIDTH, int(i*IMG_WIDTH/9)), color, 5, 1)
    return imgSolved

In [44]:
def overlayAnswer(img, imgSolved, height, width, pts1, pts2):
    matrix = cv.getPerspectiveTransform(pts2, pts1)
    imgPerspective = cv.warpPerspective(imgSolved, matrix, (width, height))
    cv.imshow("P", imgPerspective)
    imgPerspective = np.asarray(imgPerspective, np.uint8)
    imgOverlayed = cv.addWeighted(imgPerspective, 2, img, 0.8, 1)
    return imgOverlayed

In [47]:
grid = np.ones((9,9))
try:
    img = cv.imread(r'test\5.jpg')
    height, width, _ = img.shape
    img = cv.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    imgBlur = cv.GaussianBlur(imgGray,(5, 5), 3)
    imgThreshold = cv.adaptiveThreshold(imgBlur, 255, 1, 1, 11, 2)
    img_contours = img.copy()
except:
    print('File Not Found')

try:
    img_big_contours = img.copy()

    contours, hierarchy = cv.findContours(imgThreshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cv.drawContours(img_contours, contours, -1, (0, 255, 0), 3)

    biggest = biggestContour(contours)
    cv.drawContours(img_big_contours, biggest, -1, (0, 255, 0), 1)
    biggest = np.array(reorder(biggest)).reshape(8,1)    
    imgBinary, pts1, pts2 = getBinary(img, biggest)


    kernel = np.ones((6, 6),np.uint8)
    imgBinary = cv.morphologyEx(imgBinary,cv.MORPH_OPEN, kernel)

    img2grid(imgBinary, grid)
    mask = np.where(grid == 0, 1, 0)

    if solveSudoku(grid):
        imgSolved = showAnswer(mask * grid)
        cv.imshow("s", imgSolved)
        imgOverlayed = overlayAnswer(img, imgSolved, IMG_WIDTH, IMG_HEIGHT, pts1, pts2)

    else:
        print('Sudoku Not Solvable')
        print(grid)
    cv.imshow('Answer', imgBinary)
    cv.waitKey(10000)
    cv.destroyAllWindows()
    print('Sudoku Solved')
except:
    print('No Grid Detected')

Sudoku Solved


In [ ]:
# WEBCAM

In [ ]:
# while True:
#     grid = np.ones((9,9))
#     success, img = cap.read()
#     try:
#         height, width, _ = img.shape
#         img = center_crop(img, img.shape)
#         img = cv.resize(img, (IMG_WIDTH, IMG_HEIGHT))
#         imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
#         imgBlur = cv.GaussianBlur(imgGray,(5, 5), 3)
#         imgThreshold = cv.adaptiveThreshold(imgBlur, 255, 1, 1, 11, 2)
#         img_contours = img.copy()

#         contours, hierarchy = cv.findContours(imgThreshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
#         cv.drawContours(img_contours, contours, -1, (0, 255, 0), 3)

#         biggest = biggestContour(contours)
#         cv.drawContours(img_big_contours, biggest, -1, (0, 255, 0), 1)
#         biggest = np.array(reorder(biggest)).reshape(8,1)    
#         imgBinary, pts1, pts2 = getBinary(img, biggest)


#         kernel = np.ones((6, 6),np.uint8)
#         imgBinary = cv.morphologyEx(imgBinary,cv.MORPH_OPEN, kernel)

#         img2grid(imgBinary, grid)
#         mask = np.where(grid == 0, 1, 0)

#         if solveSudoku(grid):
#             imgSolved = showAnswer(mask * grid)
#             imgOverlayed = overlayAnswer(img, imgSolved, IMG_WIDTH, IMG_HEIGHT, pts1, pts2)
#         else:
#             print('Sudoku Not Solvable')
#             print(grid)
#     except:
#         pass
    
#     cv.imshow('Webcam', img)
#     if cv.waitKey(10) & 0xff == ord('q'):   
#         cv.destroyAllWindows()
#         break